# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    p_total = []
    total_invested = sum(P["Amount Invested"])
    
    for x in P.index:
        row = P.iloc[x]
        p_total.append(row.loc["Amount Invested"] / total_invested)
    
    beta = 0
    
    for x, p in enumerate(p_total):
        row = P.iloc[x]
        beta += p * row.loc["Beta"]
    
    expected = rf + beta * mkt_rp
    return expected

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [5]:
def volatility(P, cor):
    p_total = []
    total_invested = sum(P["Amount Invested"])
    
    for x in P.index:
        row = P.iloc[x]
        p_total.append(row.loc["Amount Invested"] / total_invested)
    
    var = 0
    for x, p in enumerate(p_total):
        row = P.iloc[x]
        var += (p ** 2) * (row.loc["Volatility"] ** 2)
    
    for x in range(len(p_total)):
        for y in range(x + 1, len(p_total)):
            var += 2 * p_total[x] * p_total[y] * P.iloc[x]["Volatility"] * P.iloc[y]["Volatility"] * cor
    
    
    return var ** (1/2)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [7]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    beta = vol / mkt_vol
    expected = rf + beta * mkt_rp
    return expected


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [9]:
def find_beta_2(df):
    a = df["RET"] - df["RF"]
    b = df["Mkt-RF"] - df["RF"]
    corr_df = pd.DataFrame(data = {"Ri - Rf": a.tolist(), "Rmkt - Rf": b.tolist()})
    corr_df = corr_df[["Ri - Rf", "Rmkt - Rf"]]
    corr = corr_df.corr().iloc[0, 1]
    vol = corr_df.std()
    vol_a = vol["Ri - Rf"]
    vol_b = vol["Rmkt - Rf"]
    beta = corr * vol_a / vol_b

    return beta
    

In [10]:
def CAPM_beta(return_path, factors_path):
    df1 = pd.read_csv(return_path)
    df2 = pd.read_csv(factors_path)
    df2["Mkt-RF"] = df2["Mkt-RF"].apply(lambda x: x / 100)
    df2["RF"] = df2["RF"].apply(lambda x: x / 100)
    
    df1["date"] = df1["year"].apply(lambda x : x * 100) + df1["month"]
    ndf = pd.merge(df1, df2, how='inner', on='date')
    
    test = ndf.groupby(by=["CUSIP"]).apply(find_beta_2)
    
    grouped1 = df1.groupby(by=["CUSIP"])
    avg = grouped1.mean()
    
    #Write your code here
    output = pd.DataFrame(data = {
        "CUSIP" : avg.index,
        "EXP_RETURN" : avg["RET"].tolist(),
        "BETA" : test.squeeze().tolist() # grouped2.squeeze().tolist(),
    })
    
    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [12]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    return np.polyfit(data["BETA"], data["EXP_RETURN"], 1)[1]